#  Calculating the Ne Spectral Lines Ratios and join with the results from CIGALE

-------------------

Import of the libraries

In [1]:
import numpy as np
import astropy.units as u

from numpy import unique as uniq
from astropy.table import Table, Column, table, vstack, join
from pathlib import Path
from matplotlib import pyplot as plt

from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
display(HTML("<style>.container { font-size:18px !important; }</style>"))

Reading the different paths where the results has been saved

In [2]:
path_raw = '../Datos/Raw/'
path_phot_ned = '../Datos/Work/Phot_Tables_NED/' 
path_phot_cds = '../Datos/Work/Phot_Tables_CDS/'
path_SED = '../Datos/Work/SEDs/'
path_CIGALE_phot = '../Datos/Work/CIGALE_InputPhot/'
path_CIGALE_out = '../Datos/Work/CIGALE_Output/'
path_graphs = '../Datos/Final/Graphics/'
path_FINAL = '../Datos/Final/'

Reading the table with the galaxies where Ne lines info was founded (Notebook 01)

In [3]:
date_sample = 'Sep_02_2022'
Ne_Sy = Table.read(path_raw+'Obj_Sy_Ne-IR_'+date_sample+'.vot',
                   format='votable')
print('Number of Seyfert Galaxies with Ne lines info:',len(Ne_Sy))

Number of Seyfert Galaxies with Ne lines info: 303


We remove the galaxies without energy balance that was removed in the CIGALE's phot files (Notebook 04).

In [4]:
CIG_BadGalaxies = Table.read('../Datos/Complementary/CIGPhot_BadEnergyBalance.tbl',
                             format='ascii')
for id in CIG_BadGalaxies['id']:
    Ne_Sy.remove_rows(np.where(Ne_Sy['Main_Id'] == id)[0])
print('Number of Seyfert Galaxies with Ne lines info after remove bad galaxies:',
      len(Ne_Sy))

Number of Seyfert Galaxies with Ne lines info after remove bad galaxies: 252


Now we classify the tables according to their Ne line information

In [6]:
NeV_NeII_Sy = vstack([Ne_Sy[Ne_Sy['NED_Ne_IR_info']=='NeII|NeIII|NeV'],
                      Ne_Sy[Ne_Sy['NED_Ne_IR_info']=='NeII|NeV']])

NeV_NeIII_Sy = vstack([Ne_Sy[Ne_Sy['NED_Ne_IR_info']=='NeII|NeIII|NeV'],
                       Ne_Sy[Ne_Sy['NED_Ne_IR_info']=='NeIII|NeV']])

NeIII_NeII_Sy = vstack([Ne_Sy[Ne_Sy['NED_Ne_IR_info']=='NeII|NeIII|NeV'],
                        Ne_Sy[Ne_Sy['NED_Ne_IR_info']=='NeII|NeIII']])

print('Total objects with [NeV] and [NeII] info:',len(NeV_NeII_Sy))
print('Total objects with [NeV] and [NeIII] info:',len(NeV_NeIII_Sy))
print('Total objects with [NeIII] and [NeII] info:',len(NeIII_NeII_Sy))

Total objects with [NeV] and [NeII] info: 162
Total objects with [NeV] and [NeIII] info: 156
Total objects with [NeIII] and [NeII] info: 191


------------------------------
## Function for extracting the Neon lines flux values from NED phot tables

In [7]:
def Neon_info(NED_table, Freq):
    
    '''
    This function creates a new table from the Phot NED table
    but only with the flux of [NeII] and [NeV]
    '''
    
    NED_table.remove_rows(np.where(NED_table['Flux_Density'].mask)[0])
    NED_table.remove_rows(np.where(NED_table['NED_Units'] == 'Jy')[0])
    NED_table.remove_rows(np.where(NED_table['Observed_Passband'] == '[Cl II] line (IRS)')[0])
    NED_table.remove_rows(np.where(NED_table['Observed_Passband'] == '[Cl II] 14.4 (IRS)')[0])
    NED_table.remove_rows(np.where(NED_table['Observed_Passband'] == '[Cl II] (IRS)')[0])
    NED_table.remove_rows(np.where(NED_table['Observed_Passband'] == '[Cl II] 14.37 (IRS)')[0])
    
    Ne_idx = [np.where(NED_table['Frequency'] == freq)[0] for freq in Freq]
    #print(Ne_idx)
    #print(len(Ne_idx))
    i = []
    Ne_table = Table()
    for idx in range(len(Freq)):
        Ne_table = vstack([Ne_table,NED_table[Ne_idx[idx]]])
    
    for ind in range(len(Ne_table)):
        i.append(ind)
    
    Ne_table['idx'] = i
    Ne_inf = Ne_table['idx','Observed_Passband','Flux_Density','NED_Uncertainty','NED_Units','Refcode']
    obs_passb = np.unique(Ne_inf['Observed_Passband'])
    #print(obs_passb)
    return Ne_inf

--------------------------
## Function for calculating the Neon spectral line ratios:

The inputs for the next function are the Table with the info from NED Database of each object that contains the lines info to calculate the ratio. The other inputs are the name of the passband of the lines.

For the output: 

- `line_info`: 
    - `line_info[0]`: Flux Densitys for the line 1
    - `line_info[1]`: Flux Density errors for the line 1
    - `line_info[2]`: Flux Densitys for the line 2
    - `line_info[3]`: Flux Density errors for the line 2
        
- `line_avg`: 
    - `line_avg[0]`: Weighted average for line 1 
    - `line_avg[1]`: Propagated error for line 1 weighted average 
    - `line_avg[2]`: Weighted average for line 2 
    - `line_avg[3]`: Propagated error for line 2 weighted average 
        
- `ratio_info`: 
    - `ratio_info[0]`: Ratio values of line1/line2 
    - `ratio_info[1]`: Propagated error for line ratio 
         
- `ratios_avg`: 
    - `ratios_avg[0]`: Weighted average for line ratio
    - `ratios_avg[1]`: Propagated error for line ratio weighted average 
         
- `filter_`: Name of the passbands of each line 
     
- `refcode`: Bibcodes for every line 
     
- `not_same_bib`: Number of galaxias that have the line infos but not with same bibcodes
     

In [8]:
def lineratios(NED_info,l1,l2,l1a,l2a):
    
    '''
    This function is for compute the ratio between two Neon
    emission lines (line1/line2) that have the same bibcode
    '''
    print('No. of [Ne] data',len(NED_info),'\n')
    not_same_bib = 0 #Counter of galaxies with no same bib info
    #Extract the number value of the NED_Uncertainty
    for u in range(len(NED_info)):
        #print(u,NED_info['NED_Uncertainty'][u])
        if NED_info['NED_Uncertainty'][u] != '':
            NED_info['NED_Uncertainty'][u] = float(NED_info['NED_Uncertainty'][u].split('+/-')[-1])
    
    bibcod, count = np.unique(NED_info['Refcode'],return_counts=True)#Extract the bibcodes
    flag = 0#Flag to detect when there is no info of the two Ne lines for the same bibcode
    #Print the lines' passband names
    #print('Neon line 1:',l1,'& Neon line 2:',l2)
    #print('Others filter_ names for Neon line1:',l1a,'& Neon line 2:',l2a)
    
    l1val = [] #Here we store the flux values fo the line 1
    l2val = [] #Here we store the flux values fo the line 2
    l1val_err = []
    l2val_err = []
    ratio = [] #Here we store the ratio between line1/line2
    ratio_err = []
    filter_ = []
    refcode = []
    print(NED_info,'\n')#Print the Ne Info of each galaxy
    
    for idx,bib in enumerate(bibcod):
        i = np.where(NED_info['Refcode'] == bib)[0]
        #The ratio [Ne]line1/[Ne]line2 is calculated for only the bibcodes that have both flux data
        if len(i)>=2:
            flag += 1
            #print(len(i))
            line1_i = 'nan'
            line2_i = 'nan'
            for j,ind in enumerate(i):
                if l1 in NED_info['Observed_Passband'][ind] or l1a in NED_info['Observed_Passband'][ind]:
                    #print('yes:',l1)
                    line1_i = ind
                elif l2 in NED_info['Observed_Passband'][ind] or l2a in NED_info['Observed_Passband'][ind]:
                    #print('yes:',l2)
                    line2_i = ind
            if line1_i != 'nan' and line2_i != 'nan':
                line1 = NED_info['Flux_Density'][line1_i]
                line1_err = NED_info['NED_Uncertainty'][line1_i]
                line2 = NED_info['Flux_Density'][line2_i]
                line2_err = NED_info['NED_Uncertainty'][line2_i]
                line1_line2 = line1/line2#Calculating Ne ratio per bibcode
               
                ratio.append(line1_line2)
                l1val.append(line1) #Value of the flux in line 1
                l2val.append(line2) #Value of the flux in line 2
                filter_.append(NED_info['Observed_Passband'][line1_i])
                filter_.append(NED_info['Observed_Passband'][line2_i])
                refcode.append(bib)
                #print('For bibcode',',',bib,l1,'index is:',line1_i,'and',l2,'index is:',line2_i)
                #print('For bibcode', bib,l1,'/',l2,'ratio is:',line1_line2)
                
                #Calculation of the propagation error of line1/line2
                if line1_err != '' or line2_err != '':
                    line1_line2_err = np.absolute(line1_line2)*np.sqrt((line1_err/line1)**2+(line2_err/line2)**2)
                else:
                    line1_line2_err = np.nan #Using np.nan for doing math operations
                    line1_err = np.nan
                    line2_err = np.nan
                l1val_err.append(line1_err)
                l1vals_err = np.nan_to_num(l1val_err) #Set the num values to zero to calculate the error of line 1 average
                l2val_err.append(line2_err)
                l2vals_err = np.nan_to_num(l2val_err) #Set the num values to zero to calculate the error of line 2 average
                ratio_err.append(line1_line2_err)
                ratios_err = np.nan_to_num(ratio_err) #Set the num values to zero to calculate the error of ratio average
            
            #In some cases there are two data for one bibcode but for the same spectral line
            elif (line1_i == 'nan' and line2_i != 'nan') and len(NED_info)==2:
                flag = 0
    if flag<1:
        not_same_bib += 1
        #for r in range(len(NeV_NeII_inf)): refcode.append(NeV_NeII_inf['Refcode'][r])
    
    #Calculating the average of the lines, lines ratio and its error
    values, counts = uniq(ratio, return_counts=True)
    l1values, l1counts = uniq(l1val, return_counts=True)
    l2values, l2counts = uniq(l2val, return_counts=True)
    if len(values) > 0: 
        avg_ratios = np.average(values, weights=counts)
        std_ratios = np.sqrt(np.sum(np.array(ratios_err)**2))
        avg_l1 = np.average(l1values, weights=l1counts)
        std_l1 = np.sqrt(np.sum(np.array(l1vals_err)**2))
        avg_l2 = np.average(l2values, weights=l2counts)
        std_l2 = np.sqrt(np.sum(np.array(l2vals_err)**2))
    else:
        avg_ratios = np.nan
        std_ratios = np.nan
        avg_l1 = np.nan
        std_l1 = np.nan
        avg_l2 = np.nan
        std_l2 = np.nan
    if std_ratios == 0: std_ratios = np.nan
    
    line_info = [l1val,l1val_err,l2val,l2val_err]
    line_avg = [avg_l1,std_l1,avg_l2,std_l2]
    ratio_info = [ratio,ratio_err]
    ratios_avg = [avg_ratios,std_ratios]
    
    return line_info,line_avg,ratio_info,ratios_avg,filter_,refcode,not_same_bib

In the galaxies where there are more than one data of [NeII] or [NeV], we remove the rows if per bibcode there is data for only one the them. The idea is to have both [NeII] and [NeV] data per bibcode:

## [NeV]/[NeII]

In [9]:
Nefilter = []
Refcode = []
NeV_NeII = Table(names=('Main_id','RA','DEC','otype','redshift','otypes',
                        r'[NeII]*',r'[NeII]_err*',r'[NeV]*',r'[NeV]_err*',
                        r'[NeV]/[NeII]*',r'[NeV]/[NeII]_err*','NED_Ne_IR_info'),
                 dtype=('O','f8','f8','O','f8','O','f8','f8','f8','f8','f8','f8','O'))
NeV_NeII_bib = Table(names=('Main_id','RA','DEC','otype','redshift','otypes','[NeV]/[NeII]','[NeV]/[NeII]_err',
                            'Bibcode'),
                dtype=('O','f8','f8','O','f8','O','f8','f8','O'))

not_same_bib = 0 #After call the function, it will save the number of galaxies with not same bibcode data

for id in range(len(NeV_NeII_Sy)):
    NED_phot = Table()
    obj = NeV_NeII_Sy['Main_Id'][id]
    ra = NeV_NeII_Sy['RA'][id]
    dec = NeV_NeII_Sy['DEC'][id]
    otype = NeV_NeII_Sy['otype_txt'][id]
    z = NeV_NeII_Sy['redshift'][id]
    otypes = NeV_NeII_Sy['otypes'][id]
    NED_info = NeV_NeII_Sy['NED_Ne_IR_info'][id]
    print(id)
    print('Galaxy: ',obj)

    NED_phot = Table.read(path_phot_ned+obj+'_NED_phot_tables'+'.vot', format='votable')
    print('Phot table lenght:',len(NED_phot))
    
    NeV_NeII_Freq = [20900000000000.0,21000000000000.0,23400000000000.0]
    NeV_NeII_inf = Neon_info(NED_phot, NeV_NeII_Freq)
    
    Neon,Neon_avg,Neon_ratios,avg_Neon_ratios,Ne_filter,refcode,not_same_ref = \
    lineratios(NeV_NeII_inf,'[Ne V]','[Ne II]','nan','nan')
    
    not_same_bib += not_same_ref
    
    print(f"{'[NeV] (Jy)':^18} {'[NeII] (Jy)':^14} {'[NeV]/[NeII]':^16} {'[NeV]/[NeII]_err':^19} {'Bibcode':^22}")
    
    for i in range(len(Neon[0])):
        print(f"{'%.1f'%Neon[0][i]:^18} {'%.1f'%Neon[2][i]:8} {'%g'%Neon_ratios[0][i]:^15} \
        {'%g'%Neon_ratios[1][i]:^19} {'%s'%refcode[i]:^12}")
    
    for r in range(len(refcode)): Refcode.append(refcode[r])
    for f in range(len(Ne_filter)): Nefilter.append(Ne_filter[f])
    
    NeV_NeII_row = [obj,ra,dec,otype,z,otypes,Neon_avg[2],Neon_avg[3],Neon_avg[0],Neon_avg[1],avg_Neon_ratios[0],avg_Neon_ratios[1],NED_info]
    NeV_NeII.add_row(vals=NeV_NeII_row)
    
    for b,ratio in enumerate(Neon_ratios[0]):
        NeV_NeII_bib_row = [obj,ra,dec,otype,z,otypes,ratio,Neon_ratios[1][b],refcode[b]]
        NeV_NeII_bib.add_row(vals=NeV_NeII_bib_row)
        
    print('\nWeighted Averages:')
    print('[NeV]/[NeII]:',avg_Neon_ratios[0])
    print('[NeV]/[NeII]_err:',avg_Neon_ratios[1])
    print('[NeV]:',Neon_avg[0])
    print('[NeII]:',Neon_avg[2])
    print('\n')

0
Galaxy:  3C 234.0
Phot table lenght: 253
No. of [Ne] data 4 

idx  Observed_Passband   Flux_Density ... NED_Units       Refcode      
                              Jy      ...                              
--- -------------------- ------------ ... --------- -------------------
  0   [Ne V] 14.32 (IRS) 2350000000.0 ...     Jy-Hz 2010ApJ...709.1257T
  1  [Ne V] 14.3 microns 3300000000.0 ...     Jy-Hz 2006ApJ...647..161O
  2   [Ne II] 12.8 (IRS) 1400000000.0 ...     Jy-Hz 2009ApJ...697..182T
  3 [Ne II] 12.81 micron  800000000.0 ...     Jy-Hz 2006ApJ...647..161O 

    [NeV] (Jy)      [NeII] (Jy)     [NeV]/[NeII]    [NeV]/[NeII]_err          Bibcode        
   3300000000.0    800000000.0      4.125                    1.35244       2006ApJ...647..161O

Weighted Averages:
[NeV]/[NeII]: 4.125
[NeV]/[NeII]_err: 1.3524428130239001
[NeV]: 3300000000.0
[NeII]: 800000000.0


1
Galaxy:  NGC   262
Phot table lenght: 334
No. of [Ne] data 7 

idx  Observed_Passband   Flux_Density ... NED_Units      

In [10]:
print('Number of galaxies with diferent bibcode for [NeV] and [NeII] fluxes:',not_same_bib)
print('Galaxies idx with diferent bibcode for [NeV] and [NeII] fluxes:',\
      np.where(np.isnan(NeV_NeII[r'[NeV]/[NeII]*']))[0])

#Uncomment the 'for' lines for print the 'Observed_passband'and the Ŕefcodes' with [NeV] and [NeII] information
Ne_filters1, countspb1 = np.unique(Nefilter, return_counts=True)
#for pb in range(len(Ne_filters1)): print(Ne_filters1[pb])
    
refcodes1, countsrc1 = np.unique(Refcode, return_counts=True)
#for rc in range(len(refcodes1)): print(refcodes1[rc])

Number of galaxies with diferent bibcode for [NeV] and [NeII] fluxes: 4
Galaxies idx with diferent bibcode for [NeV] and [NeII] fluxes: [ 31 110 118 151]


In [11]:
#Remove the galaxies where there was not info with same bibcode
NeV_NeII.remove_rows(np.where(np.isnan(NeV_NeII[r'[NeV]/[NeII]*']))[0])
NeV_NeII_bib.remove_rows(np.where(np.isnan(NeV_NeII[r'[NeV]/[NeII]*']))[0])

print('Final number of galaxies with [Ne V] and [Ne II] info: ', len(NeV_NeII))

Final number of galaxies with [Ne V] and [Ne II] info:  158


## [NeV]/[NeIII]

In [12]:
Nefilter = []
Refcode = []
NeV_NeIII = Table(names=('Main_id','RA','DEC','otype','redshift','otypes',
                         r'[NeV]$^+$',r'[NeV]_err$^+$',r'[NeIII]$^+$',r'[NeIII]_err$^+$',
                         r'[NeV]/[NeIII]$^+$',r'[NeV]/[NeIII]_err$^+$','NED_Ne_IR_info'),
                dtype=('O','f8','f8','O','f8','O','f8','f8','f8','f8','f8','f8','O'))
NeV_NeIII_bib = Table(names=('Main_id','RA','DEC','otype','redshift','otypes','[NeV]/[NeIII]','[NeV]/[NeIII]_err',
                             'Bibcode'),
                dtype=('O','f8','f8','O','f8','O','f8','f8','O'))

not_same_bib = 0 #After call the function, it will save the number of galaxies with not same bibcode data

for id in range(len(NeV_NeIII_Sy)):
    NED_phot = Table()
    obj = NeV_NeIII_Sy['Main_Id'][id]
    ra = NeV_NeIII_Sy['RA'][id]
    dec = NeV_NeIII_Sy['DEC'][id]
    otype = NeV_NeIII_Sy['otype_txt'][id]
    z = NeV_NeIII_Sy['redshift'][id]
    otypes = NeV_NeIII_Sy['otypes'][id]
    NED_info = NeV_NeIII_Sy['NED_Ne_IR_info'][id]
    print(id)
    print('Galaxy: ',obj)

    NED_phot = Table.read(path_phot_ned+obj+'_NED_phot_tables'+'.vot', format='votable')
    print('Phot table lenght:',len(NED_phot))
    
    NeV_NeIII_Freq = [20900000000000.0,21000000000000.0,19200000000000.0,19300000000000.0]
    NeV_NeIII_inf = Neon_info(NED_phot, NeV_NeIII_Freq)
    
    Neon,Neon_avg,Neon_ratios,avg_Neon_ratios,Ne_filter,refcode,not_same_ref = \
    lineratios(NeV_NeIII_inf,'[Ne V]','[Ne III]','nan','[NeIII]')
    
    not_same_bib += not_same_ref
    
    print(f"{'[NeV] (Jy)':^18} {'[NeIII] (Jy)':^14} {'[NeV]/[NeIII]':^16} {'[NeV]/[NeIII]_err':^19} {'Bibcode':^22}")
    
    for i in range(len(Neon[0])):
        print(f"{'%.1f'%Neon[0][i]:^18} {'%.1f'%Neon[2][i]:8} {'%g'%Neon_ratios[0][i]:^15} {'%g'%Neon_ratios[1][i]:^19} {'%s'%refcode[i]:^12}")
    
    for r in range(len(refcode)): Refcode.append(refcode[r])
    for f in range(len(Ne_filter)): Nefilter.append(Ne_filter[f])
    
    NeV_NeIII_row = [obj,ra,dec,otype,z,otypes,Neon_avg[0],Neon_avg[1],Neon_avg[2],Neon_avg[3],avg_Neon_ratios[0],avg_Neon_ratios[1],NED_info]
    NeV_NeIII.add_row(vals=NeV_NeIII_row)
    
    for b,ratio in enumerate(Neon_ratios[0]):
        NeV_NeIII_bib_row = [obj,ra,dec,otype,z,otypes,ratio,Neon_ratios[1][b],refcode[b]]
        NeV_NeIII_bib.add_row(vals=NeV_NeIII_bib_row)
        
    print('\nAverage [NeV]/[NeIII]:',avg_Neon_ratios[0])
    print('Average Error:',avg_Neon_ratios[1])
    print('Average [NeV]:',Neon_avg[0])
    print('Average [NeIII]:',Neon_avg[2])
    print('\n')

0
Galaxy:  3C 234.0
Phot table lenght: 253
No. of [Ne] data 4 

idx  Observed_Passband   Flux_Density ... NED_Units       Refcode      
                              Jy      ...                              
--- -------------------- ------------ ... --------- -------------------
  0   [Ne V] 14.32 (IRS) 2350000000.0 ...     Jy-Hz 2010ApJ...709.1257T
  1  [Ne V] 14.3 microns 3300000000.0 ...     Jy-Hz 2006ApJ...647..161O
  2 [Ne III] 15.56 (IRS) 3400000000.0 ...     Jy-Hz 2010ApJ...709.1257T
  3 [Ne III] 15.55 micrn 8200000000.0 ...     Jy-Hz 2006ApJ...647..161O 

    [NeV] (Jy)      [NeIII] (Jy)   [NeV]/[NeIII]    [NeV]/[NeIII]_err         Bibcode        
   3300000000.0    8200000000.0    0.402439          0.0920194      2006ApJ...647..161O
   2350000000.0    3400000000.0    0.691176          0.100915       2010ApJ...709.1257T

Average [NeV]/[NeIII]: 0.5468077474892397
Average Error: 0.13657008334636164
Average [NeV]: 2825000000.0
Average [NeIII]: 5800000000.0


1
Galaxy:  NGC   262
P

In [13]:
print('Number of galaxies with diferent bibcode for [NeV] and [NeIII] fluxes:',not_same_bib)
print('Galaxies idx with diferent bibcode for [NeV] and [NeIII] fluxes:',\
      np.where(np.isnan(NeV_NeIII[r'[NeV]/[NeIII]$^+$']))[0])

#Uncomment the 'for' lines for print the 'Observed_passband'and the Ŕefcodes' with [NeV] and [NeIII] information
Ne_filters2, countspb2 = np.unique(Nefilter, return_counts=True)
#for pb in range(len(Ne_filters2)): print(Ne_filters2[pb])
    
refcodes2, countsrc2 = np.unique(Refcode, return_counts=True)
#for rc in range(len(refcodes2)): print(refcodes2[rc])

Number of galaxies with diferent bibcode for [NeV] and [NeIII] fluxes: 3
Galaxies idx with diferent bibcode for [NeV] and [NeIII] fluxes: [ 31  56 118]


In [14]:
#Remove the galaxies where there was not info with same bibcode
NeV_NeIII.remove_rows(np.where(np.isnan(NeV_NeIII[r'[NeV]/[NeIII]$^+$']))[0])
NeV_NeIII_bib.remove_rows(np.where(np.isnan(NeV_NeIII[r'[NeV]/[NeIII]$^+$']))[0])

print('Final number of galaxies with [Ne V] and [Ne III] info: ', len(NeV_NeIII))

Final number of galaxies with [Ne V] and [Ne III] info:  153


## [NeIII]/[NeII]

In [15]:
Nefilter = []
Refcode = []
NeIII_NeII = Table(names=('Main_id','RA','DEC','otype','redshift','otypes',
                          r'[NeII]$^-$',r'[NeII]_err$^-$',r'[NeIII]$^-$',r'[NeIII]_err$^-$',
                          r'[NeIII]/[NeII]$^-$',r'[NeIII]/[NeII]_err$^-$','NED_Ne_IR_info'),
                dtype=('O','f8','f8','O','f8','O','f8','f8','f8','f8','f8','f8','O'))
NeIII_NeII_bib = Table(names=('Main_id','RA','DEC','otype','redshift','otypes','[NeIII]/[NeII]','[NeIII]/[NeII]_err',
                              'Bibcode'),
                dtype=('O','f8','f8','O','f8','O','f8','f8','O'))

not_same_bib = 0 #After call the function, it will save the number of galaxies with not same bibcode data

for id in range(len(NeIII_NeII_Sy)):
    NED_phot = Table()
    obj = NeIII_NeII_Sy['Main_Id'][id]
    ra = NeIII_NeII_Sy['RA'][id]
    dec = NeIII_NeII_Sy['DEC'][id]
    otype = NeIII_NeII_Sy['otype_txt'][id]
    z = NeIII_NeII_Sy['redshift'][id]
    otypes = NeIII_NeII_Sy['otypes'][id]
    NED_info = NeIII_NeII_Sy['NED_Ne_IR_info'][id]
    print(id)
    print('Galaxy: ',obj)

    NED_phot = Table.read(path_phot_ned+obj+'_NED_phot_tables'+'.vot', format='votable')
    print('Phot table lenght:',len(NED_phot))
    
    NeIII_NeII_Freq = [19200000000000.0,19300000000000.0,23400000000000.0]
    NeIII_NeII_inf = Neon_info(NED_phot, NeIII_NeII_Freq)
    
    Neon,Neon_avg,Neon_ratios,avg_Neon_ratios,Ne_filter,refcode,not_same_ref = \
    lineratios(NeIII_NeII_inf,'[Ne III]','[Ne II]','[NeIII]','nan')
    
    not_same_bib += not_same_ref
    
    print(f"{'[NeIII] (Jy)':^18} {'[NeII] (Jy)':^14} {'[NeIII]/[NeII]':^16} {'[NeIII]/[NeII]_err':^19} {'Bibcode':^22}")
    
    for i in range(len(Neon[0])):
        print(f"{'%.1f'%Neon[0][i]:^18} {'%.1f'%Neon[2][i]:8} {'%g'%Neon_ratios[0][i]:^15} {'%g'%Neon_ratios[1][i]:^19} {'%s'%refcode[i]:^12}")
    
    for r in range(len(refcode)): Refcode.append(refcode[r])
    for f in range(len(Ne_filter)): Nefilter.append(Ne_filter[f])
    
    NeIII_NeII_row = [obj,ra,dec,otype,z,otypes,Neon_avg[2],Neon_avg[3],Neon_avg[0],Neon_avg[1],avg_Neon_ratios[0],avg_Neon_ratios[1],NED_info]
    NeIII_NeII.add_row(vals=NeIII_NeII_row)
    
    for b,ratio in enumerate(Neon_ratios[0]):
        NeIII_NeII_bib_row = [obj,ra,dec,otype,z,otypes,ratio,Neon_ratios[1][b],refcode[b]]
        NeIII_NeII_bib.add_row(vals=NeIII_NeII_bib_row)
        
    print('\nAverage [NeIII]/[NeII]:',avg_Neon_ratios[0])
    print('Average Error:',avg_Neon_ratios[1])
    print('Average [NeIII]:',Neon_avg[0])
    print('Average [NeII]:',Neon_avg[2])
    print('\n')

0
Galaxy:  3C 234.0
Phot table lenght: 253
No. of [Ne] data 4 

idx  Observed_Passband   Flux_Density ... NED_Units       Refcode      
                              Jy      ...                              
--- -------------------- ------------ ... --------- -------------------
  0 [Ne III] 15.56 (IRS) 3400000000.0 ...     Jy-Hz 2010ApJ...709.1257T
  1 [Ne III] 15.55 micrn 8200000000.0 ...     Jy-Hz 2006ApJ...647..161O
  2   [Ne II] 12.8 (IRS) 1400000000.0 ...     Jy-Hz 2009ApJ...697..182T
  3 [Ne II] 12.81 micron  800000000.0 ...     Jy-Hz 2006ApJ...647..161O 

   [NeIII] (Jy)     [NeII] (Jy)    [NeIII]/[NeII]  [NeIII]/[NeII]_err         Bibcode        
   8200000000.0    800000000.0      10.25            2.70777       2006ApJ...647..161O

Average [NeIII]/[NeII]: 10.25
Average Error: 2.7077723778043086
Average [NeIII]: 8200000000.0
Average [NeII]: 800000000.0


1
Galaxy:  NGC   262
Phot table lenght: 334
No. of [Ne] data 8 

idx  Observed_Passband    Flux_Density ... NED_Units       

In [16]:
print('Number of galaxies with diferent bibcode for [NeIII] and [NeII] fluxes:',not_same_bib)
print('Galaxies idx with diferent bibcode for [NeIII] and [NeII] fluxes:',\
      np.where(np.isnan(NeIII_NeII[r'[NeIII]/[NeII]$^-$']))[0])

#Uncomment the 'for' lines for print the 'Observed_passband'and the Ŕefcodes' with [NeIII] and [NeII] information
Ne_filters3, countspb3 = np.unique(Nefilter, return_counts=True)
#for pb in range(len(Ne_filters3)): print(Ne_filters3[pb])

refcodes3, countsrc3 = np.unique(Refcode, return_counts=True)
#for rc in range(len(refcodes3)): print(refcodes3[rc])

Number of galaxies with diferent bibcode for [NeIII] and [NeII] fluxes: 1
Galaxies idx with diferent bibcode for [NeIII] and [NeII] fluxes: [110]


In [17]:
#Remove the galaxies where there was not info with same bibcode
NeIII_NeII.remove_rows(np.where(np.isnan(NeIII_NeII[r'[NeIII]/[NeII]$^-$']))[0])
NeIII_NeII_bib.remove_rows(np.where(np.isnan(NeIII_NeII[r'[NeIII]/[NeII]$^-$']))[0])

print('Final number of galaxies with [Ne III] and [Ne II] info: ', len(NeIII_NeII))

Final number of galaxies with [Ne III] and [Ne II] info:  190


----------------------------------
## Joining tables with CIGALE results:

If we want to join to the different CIGALE results after the data cleaning from Notebook 5, we must to change the file name in the next cell:

- results0a90_All.fits
- results30y70_All.fits
- results_bestChi.fits
- resultsDBIC_All.fits

In [18]:
CIGresults = Table.read(path_CIGALE_out+'resultsDBIC_All.fits', format='fits')
#CIGresults

In [19]:
CIGresults = CIGresults['id',
                        'bayes.agn.fracAGN','bayes.agn.fracAGN_err',
                        'bayes.agn.i','bayes.agn.i_err',
                        'bayes.agn.disk_luminosity','bayes.agn.disk_luminosity_err',
                        'bayes.agn.luminosity','bayes.agn.luminosity_err',
                        'bayes.sfh.sfr','bayes.sfh.sfr_err',
                        'bayes.stellar.m_star','bayes.stellar.m_star_err',
                        'best.reduced_chi_square','log.reduced_chi_square','CIGALE_setup']
CIGresults['id'] = CIGresults['id'].astype(str) #Converting the dtype to string
CIGresults['id'] = CIGresults['id'].astype(object) #Converting the dtype to object
CIGresults['id'].name = 'Main_id' #Changing the column name
len(CIGresults)

120

Joining the table with the ratio [NeV]/[NeII] info and CIGALE results:

In [20]:
NeVNeII = join(NeV_NeII, CIGresults, join_type='left', keys='Main_id')
NeVNeII = NeVNeII.filled(np.nan) #Fill the mask values (---) with numpy.nan
NeVNeII.remove_rows(np.where(np.isnan(NeVNeII['bayes.agn.i']))[0])
#NeVNeII.show_in_notebook()

Joining the table with the ratio [NeV]/[NeIII] info and CIGALE results:

In [21]:
NeVNeIII = join(NeV_NeIII, CIGresults, join_type='left', keys='Main_id')
NeVNeIII = NeVNeIII.filled(np.nan) #Fill the mask values (---) with numpy.nan
NeVNeIII.remove_rows(np.where(np.isnan(NeVNeIII['bayes.agn.i']))[0])
#NeVNeIII.show_in_notebook()

Joining the table with the ratio [NeIII]/[NeII] info and CIGALE results:

In [22]:
NeIIINeII = join(NeIII_NeII, CIGresults, join_type='left', keys='Main_id')
NeIIINeII = NeIIINeII.filled(np.nan) #Fill the mask values (---) with numpy.nan
NeIIINeII.remove_rows(np.where(np.isnan(NeIIINeII['bayes.agn.i']))[0])
#NeIIINeII.show_in_notebook()

Joinig the NeVNeII and NeVNeIII tables:

In [23]:
NeVNeIINeIII = join(NeVNeII, NeVNeIII, join_type='outer')

In [24]:
Sy_Ne = join(NeVNeIINeIII, NeIIINeII, join_type='outer')
Sy_Ne = Sy_Ne.filled(np.nan) #Fill the mask values (---) with numpy.nan

Sy_Ne = Sy_Ne['Main_id','RA','DEC','otype','redshift','otypes',
              r'[NeV]*',r'[NeV]_err*',r'[NeII]*',r'[NeII]_err*',
              r'[NeV]/[NeII]*',r'[NeV]/[NeII]_err*',
              r'[NeV]$^+$',r'[NeV]_err$^+$',r'[NeIII]$^+$',r'[NeIII]_err$^+$',
              r'[NeV]/[NeIII]$^+$',r'[NeV]/[NeIII]_err$^+$',
              r'[NeIII]$^-$',r'[NeIII]_err$^-$',r'[NeII]$^-$',r'[NeII]_err$^-$',
              r'[NeIII]/[NeII]$^-$',r'[NeIII]/[NeII]_err$^-$',
              'bayes.agn.fracAGN','bayes.agn.fracAGN_err',
              'bayes.agn.i','bayes.agn.i_err',
              'bayes.agn.disk_luminosity','bayes.agn.disk_luminosity_err',
              'bayes.agn.luminosity','bayes.agn.luminosity_err',
              'bayes.sfh.sfr','bayes.sfh.sfr_err',
              'bayes.stellar.m_star','bayes.stellar.m_star_err',
              'best.reduced_chi_square','log.reduced_chi_square',
              'CIGALE_setup','NED_Ne_IR_info'] #reorder columns

Sy_Ne.write(path_FINAL+'Ne_ratios_-_CIGALE_results.tbl', format='ascii',overwrite=True)
#Sy_Ne.write(path_FINAL+'Ne_ratios_-_CIGALE_results.fits', format='fits',overwrite=True)

-----------------------------
### By bibcode

In [25]:
NeVNeII_bib = join(NeV_NeII_bib, CIGresults, join_type='left', keys='Main_id')
NeVNeII_bib = NeVNeII_bib.filled(np.nan) #Fill the mask values (---) with numpy.nan
NeVNeII_bib.remove_rows(np.where(np.isnan(NeVNeII_bib['bayes.agn.i']))[0])
NeVNeIII_bib = join(NeV_NeIII_bib, CIGresults, join_type='left', keys='Main_id')
NeVNeIII_bib = NeVNeIII_bib.filled(np.nan) #Fill the mask values (---) with numpy.nan
NeVNeIII_bib.remove_rows(np.where(np.isnan(NeVNeIII_bib['bayes.agn.i']))[0])
NeIIINeII_bib = join(NeIII_NeII_bib, CIGresults, join_type='left', keys='Main_id')
NeIIINeII_bib = NeIIINeII_bib.filled(np.nan) #Fill the mask values (---) with numpy.nan
NeIIINeII_bib.remove_rows(np.where(np.isnan(NeIIINeII_bib['bayes.agn.i']))[0])
NeVNeIINeIII_bib = join(NeVNeII_bib, NeVNeIII_bib, join_type='outer')
Sy_Ne_bib = join(NeVNeIINeIII_bib, NeIIINeII_bib, join_type='outer')
Sy_Ne_bib = Sy_Ne_bib.filled(np.nan) #Fill the mask values (---) with numpy.nan

Sy_Ne_bib = Sy_Ne_bib['Main_id','RA','DEC','otype','redshift','otypes','[NeV]/[NeII]','[NeV]/[NeII]_err',\
              '[NeV]/[NeIII]','[NeV]/[NeIII]_err','[NeIII]/[NeII]','[NeIII]/[NeII]_err',\
              'bayes.agn.fracAGN','bayes.agn.fracAGN_err','bayes.agn.i','bayes.agn.i_err',\
              'bayes.sfh.sfr','bayes.sfh.sfr_err','bayes.stellar.m_star','bayes.stellar.m_star_err',\
              'best.reduced_chi_square','log.reduced_chi_square'] #reorder columns

Sy_Ne_bib.write(path_FINAL+'Ne_ratios_-_CIGALE_results_bib.tbl', format='ascii',overwrite=True)
#Sy_Ne_bib.show_in_notebook()

-------------------------------
## Notebook Info

In [26]:
%reload_ext watermark
%watermark -a "Jonhatan Bernal" -d -v -m
print('Specific Python packages')
%watermark -iv -w --packages astropy

Author: Jonhatan Bernal

Python implementation: CPython
Python version       : 3.12.0
IPython version      : 8.14.0

Compiler    : GCC 13.2.1 20230918 (Red Hat 13.2.1-3)
OS          : Linux
Release     : 6.5.11-300.fc39.x86_64
Machine     : x86_64
Processor   : 
CPU cores   : 16
Architecture: 64bit

Specific Python packages
astropy: 5.3.2

astropy   : 5.3.2
numpy     : 1.24.4
matplotlib: 3.8.1

Watermark: 2.4.3



In [27]:
Sy_Ne.show_in_notebook()

idx,Main_id,RA,DEC,otype,redshift,otypes,[NeV]*,[NeV]_err*,[NeII]*,[NeII]_err*,[NeV]/[NeII]*,[NeV]/[NeII]_err*,[NeV]$^+$,[NeV]_err$^+$,[NeIII]$^+$,[NeIII]_err$^+$,[NeV]/[NeIII]$^+$,[NeV]/[NeIII]_err$^+$,[NeIII]$^-$,[NeIII]_err$^-$,[NeII]$^-$,[NeII]_err$^-$,[NeIII]/[NeII]$^-$,[NeIII]/[NeII]_err$^-$,bayes.agn.fracAGN,bayes.agn.fracAGN_err,bayes.agn.i,bayes.agn.i_err,bayes.agn.disk_luminosity,bayes.agn.disk_luminosity_err,bayes.agn.luminosity,bayes.agn.luminosity_err,bayes.sfh.sfr,bayes.sfh.sfr_err,bayes.stellar.m_star,bayes.stellar.m_star_err,best.reduced_chi_square,log.reduced_chi_square,CIGALE_setup,NED_Ne_IR_info
,,,,,,,,,,,,,,,,,,,,,,,,,,,deg,deg,W,W,W,W,solMass / yr,solMass / yr,solMass,solMass,,,,
0,2C 94,17.220292429510003,13.33729522217,Sy2,0.059615,*|AGN|G|Rad|Sy2|X,2000000000.0,300000000.0,3900000000.0,200000000.0,0.5128205128205128,0.0812943431354469,2000000000.0,300000000.0,5300000000.0,200000000.0,0.37735849056603776,0.05836748377170459,5300000000.0,200000000.0,3900000000.0,200000000.0,1.358974358974359,0.08652562196640158,0.6865611565783494,0.11967341436995055,73.83482561588654,8.508248513418419,1.3787638404047036e+35,5.1210194135958615e+34,2.4468584696260853e+37,4.030152954820804e+36,1.7246193065665494,1.3163365088890544,28682725610.48474,6450427912.662605,1.3173407609701728,0.11969812990994547,0to90,NeII|NeIII|NeV
1,2C 1883,349.14663898619995,4.08841586141,Sy2,0.22,*|AGN|G|IR|QSO|Rad|Sy2,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,1000000000.0,70000000.0,3110000000.0,220000000.0,0.3215434083601286,0.03199976335108683,0.32208330842877636,0.08324033255971083,60.56433792538642,16.98218961730912,2.6106137265995487e+37,1.2836460511636971e+37,2.024602523457805e+38,4.764869450162955e+37,78.94081174225944,12.95918487733214,103980874386.1715,43864297039.03055,1.0612390066809732,0.02581320444162545,30and70,NeII|NeIII
2,2E 2389,166.0577547903,76.98283181919,Sy1,0.3115,*|BH?|G|IR|QSO|Rad|Sy1|UV|X|gam,505000000.0,59000000.0,437000000.0,40000000.0,1.1556064073226544,0.17151303566833198,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.8188987719707819,0.024258975221072464,26.011843998959925,11.071888539354369,1.8591392566232316e+39,1.255581728634208e+38,2.3603186824229648e+39,1.1801593412114825e+38,32.99957287515418,13.005716016371974,82007440862.93582,69392222513.68936,1.442908260688541,0.15923871976759285,0to90,NeII|NeV
3,2E 3648,245.09091058141,17.60665343366,Sy1,0.55517,*|BH?|G|IR|QSO|Rad|Sy1|X,1180000000.0,170000000.0,660000000.0,40000000.0,1.7878787878787878,0.279439361062646,1190000000.0,170000000.0,1610000000.0,130000000.0,0.7391975308641976,0.12011916377404842,1620000000.0,130000000.0,660000000.0,40000000.0,2.4545454545454546,0.2468333394786096,0.7373844941753334,0.09571439256469585,32.20528489575952,9.404682549643882,2.0566345877118184e+39,1.3435115805324631e+38,3.149791697935656e+39,1.574895848967828e+38,110.83038756721615,54.366157677823296,96631370793.32745,102566202983.27977,2.3766658631781086,0.37596812830862414,0to90,NeII|NeIII|NeV
4,2MASS J13000535+1632148,195.02228295163,16.53743305218,Sy1,0.07993,*|AGN|G|IR|Rad|Sy1|X,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,1450000000.0,0.0,1670000000.0,0.0,0.8682634730538922,nan,0.8997482762713642,0.0035397704007171173,60.00022899305476,0.047852752121327925,1.860024667933385e+36,9.300123339666926e+34,9.410611315882008e+37,4.705305657941004e+36,0.5592751145937703,0.3523226196938686,90494096430.65376,16511046241.395515,2.9185132854550697,0.4651616746160927,0to90,NeII|NeIII
5,2MASSI J0918486+211717,139.7025485953,21.288075035289996,Sy1,0.14903,*|AGN|IR|QSO|Sy1|X,4480000000.0,0.0,982000000.0,0.0,4.562118126272912,nan,4480000000.0,0.0,2600000000.0,0.0,1.7230769230769232,nan,2600000000.0,0.0,982000000.0,0.0,2.6476578411405294,nan,0.8988839720632542,0.013870021401598235,44.20391814907898,14.529180682596715,1.8396031516838928e+37,4.904386892756304e+36,2.495614659398097e+38,1.2478073296990486e+37,0.5329401797567834,1.1935901737550823,152823017793.68384,53131442843.11487,0

In [28]:
len(np.where(Sy_Ne['CIGALE_setup']=='0to90')[0])

90

In [29]:
len(np.where(Sy_Ne['CIGALE_setup']=='30and70')[0])

7